In [ ]:
#HCD_ksads - using new structures created by NDA to house scores only, since item level data was found to be
#inconsistently annotated.  


In [ ]:
import os, datetime
import csv
#import pycurl
import sys
import shutil
from openpyxl import load_workbook
import pandas as pd
from io import BytesIO
import numpy as np
import subprocess
from scipy import stats
#import yaml
import json

from ccf.redcap import RedcapTable
from ccf.box import LifespanBox
from ccf.config import LoadSettings

#from download.redcap import Redcap
#from download.box import LifespanBox
snapshotdate = datetime.datetime.today().strftime('%m_%d_%Y')
config = LoadSettings()


In [ ]:
box=LifespanBox(cache='./cache/')
verbose = True
snapshotdate = datetime.datetime.today().strftime('%m_%d_%Y')
#pathout="./prepped/hcd" 
pathout="."
racepath="./prepped"

#Rosetta (a.k.a Inventory) file will have all the nda vars and pedids
extrainfo=config['rosetta']['filename']
eventlist=['V1','V2','V3']
#eventlist=['visit_1_arm_1','visit_2_arm_1','visit_3_arm_1','visit_arm_1']

extrainfo

### remove dud scores for the following, per Deanna 2021 (KSADS makers were supposed to fix, but no word as of March 2022):

- Major Depression 
- Persistent Depressive Disorder
- Disruptive Mood Dysregulation Disorder
- Attention Deficit Hyperactivity Disorder
- Agoraphobia
- Anorexia

In [ ]:
b=[905784566785]
b2=box.download_files(b)

dir(b2)
b2
bvars=pd.read_excel(b2[0], sheet_name='VariablesExcluded4BadScores')
bad_drops=list(bvars['Variable / Field Name'])

bad_drops[1:10]

In [ ]:
#get data from PreRelease folder, since the MOOD batteries have been rolled in to regular batteries there, thus one row per 
#person per type
#see eof for method of getting report directly from REDCap if so desired

mask=[887050736739]
a=box.download_files(mask)

def getlist(mask,sheet):
    restrictA=pd.read_excel(mask, sheet_name=sheet)
    restrictedA=list(restrictA.field_name)
    return restrictedA


restrictedK=getlist(a[0],'ksads')


In [ ]:
restrictedK

### get data from PreRelease folder, since the MOOD batteries have been rolled in to regular batteries there, thus one row per  person per type see eof for method of getting report directly from REDCap if so desired


In [ ]:
kfile=[911850508658]
box=LifespanBox(cache='./')
k=box.download_files(kfile)

In [ ]:
k[0]
ksads = pd.read_csv(k[0],low_memory=False,dtype=str)

#already dropped
print(ksads.shape)
ksads=ksads.drop(restrictedK, axis=1, errors='ignore')
print(ksads.shape)
print(len(ksads.columns))

#drop bad scores
print(ksads.shape)
ksads=ksads.drop(bad_drops, axis=1, errors='ignore')
print(ksads.shape)
print(len(ksads.columns))


In [ ]:
#0 = The Module related to the Symptom/Diagnosis is ADMINISTERED, however the Symptom/Diagnosis is NOT PRESENT; 
#1 = The Module related to the Symptom/Diagnosis is ADMINISTERED AND the Symptom/Diagnosis is PRESENT; 
#555 = No data/modules collected for subject/event; 888 = The Module is not administered, e.g. individual did not answer screening items affirmatively and value missing due to branching logic
ksadswork=ksads.copy()#ksads[['id','patientid','additionalinfo','patienttype','k_unusable','k_unusable_specify',value,admin]].copy()
ksadswork.head()
#ksadswork.replace(np.nan, 3)

justk=ksadswork.drop(columns=['id', 'patientid', 'additionalinfo', 'patienttype', 'sitename',
       'k_unusable', 'k_unusable_specify', 'common_complete'])

for i in justk:
    ksadswork.loc[ksadswork[i].isnull()==True,i]=''
    
    
ksadswork.head()#.ksadsd17_administered.value_counts()


In [ ]:
for i in range(1,971):
    value='ksadsd'+str(i)+'_value'
    admin='ksadsd'+str(i)+'_administered'
    newadmin='ksadsd'+str(i)+'_administeredNDA'
    #ksadswork=ksads[['subject','redcap_event','id','patientid','additionalinfo','patienttype',value,admin]].copy()
    #not really necessary to split these things up into if/elif clauses, but wanted to make sure block B captured all patterns
    try:
        if value in list(justk):
            a=list(ksadswork[value].unique())
            a.sort()
            b=list(ksadswork[admin].unique())
            b.sort()
            #print(b)
            if a==['', '0', '1'] and b==['', '0', '1']:
                ksadswork[newadmin]=0
                ksadswork.loc[(ksadswork[admin]=='1') & (ksadswork[value]=='1'),newadmin]='1'
                ksadswork.loc[(ksadswork[admin]=='1') & (ksadswork[value]=='0'),newadmin]='0'
                ksadswork.loc[(ksadswork[admin]=='0'),newadmin]='888' # & (ksadswork[value]=='0'),newadmin]='0'
                ksadswork.loc[(ksadswork[admin]==''),newadmin]='' # & (ksadswork[value]=='0'),newadmin]='0'
            elif a!=['', '0', '1'] and b==['', '0', '1']:  #character value case found in block B
                if a==['','0']:
                    #print('NO ONE HAS SX or not measured ' + value + ':')
                    pass
                elif a==['0','1']:
                    #print('EVERYONE TESTED ' + value + ':')
                    pass
                else:
                     pass
                #BLOCK B for cases of values being long descriptive strings
                ksadswork[newadmin]=0
                ksadswork.loc[(ksadswork[admin]=='1') & (~(ksadswork[value].isin(['','0']))),newadmin]='1'
                ksadswork.loc[(ksadswork[admin]=='1') & (ksadswork[value]=='0'),newadmin]='0'
                ksadswork.loc[(ksadswork[admin]=='0'),newadmin]='888' 
                ksadswork.loc[(ksadswork[admin]==''),newadmin]=''
                #END BLOCK B
            else:
                print(admin)
                print(value)
                print('WARNING!!!')
                print('unexpected values or admins for '+admin+ ' and ' + value)
                print('values:'+ str(a))
                print('admins:'+ str(b))
        else: 
            print('value',value,'is not in justk')
            
    except:
        print('2. problem with ',admin,value)


ksadswork.head()#[['ksads960_administered','ksadsd960_value']].head()


In [ ]:
ksadswork.to_csv('temp.csv',index=False)
#ksadswork=pd.read_csv('temp.csv',low_memory=False)
#ksadswork=ksadswork.drop(columns='Unnamed: 0').copy()
ksadswork.head()


In [ ]:
#the NDA structure used the _administered variable to refer to the combined variable we just created, so now we have to 
#drop the old one and rename the new one.

ksadscols=list(ksads.columns)
ksadsnewcols=['id','patientid','patienttype','k_unusable']

#ksads.columns
for i in range(1,971):
    value='ksadsd'+str(i)+'_value'
    admin='ksadsd'+str(i)+'_administered'
    newadmin='ksadsd'+str(i)+'_administeredNDA'
    if value not in ksadscols:
        print(value+ ' not in columns')
    else:
        ksadsnewcols=ksadsnewcols + [newadmin]
        
#    newadmin='ksadsd'+str(i)+'_administeredNDA'

print(len(ksadsnewcols))

ksadsworknew=ksadswork#[ksadsnewcols]
#ksadsworknew.to_csv('tempnew.csv')
#print(ksadswork.shape)
#print(ksadsworknew.shape)
ksadsworknew.head()


In [ ]:
ksadsworknew.head()


In [ ]:
oldname=[]
newname=[]

for i in range(1,971):
    newadmin='ksadsd'+str(i)+'_administeredNDA'
    rename='ksadsd'+str(i)+'_administered'
    if newadmin in ksadsnewcols:
        oldname=oldname+[newadmin]
        newname=newname+[rename]
    else:
        print(newadmin + ' not in columns')
    



In [ ]:
print(len(oldname))
print(len(newname))
renames=dict(zip(oldname,newname))


In [ ]:
#ksadsnewcols
#print(len(ksadsnewcols))
#print(len(renames))
#print(ksadsworknew.shape)
#ksadsworknew.head()


print(len(ksadsworknew.columns))
ksadsnewcols2=['subject','redcap_event','id', 'patientid', 'additionalinfo', 'patienttype', 'sitename',
       'k_unusable', 'k_unusable_specify', 'common_complete']+ksadsnewcols
ksadsslim=ksadsworknew[ksadsnewcols2]
ksadsslim.head()
ksadsslim.columns[0:30]
print(len(ksadsslim.columns))
ksadsslim = ksadsslim.loc[:,~ksadsslim.columns.duplicated()]
ksadsslim.patienttype
print(len(ksadsslim.columns))

#print(len(df.columns))

In [ ]:
#renames
ksadsslim=ksadsslim.rename(columns=renames).copy()
ksadsslim.columns[-10:]
ksadsslim.columns[0:25]

In [ ]:
#pull in the ndar fields
extrainfo
#rosetta=pd.read_csv(extrainfo)[['DB_Source','REDCap_id','subject','pseudo_guid','nda_interview_date','nda_age','M/F','pedid','redcap_event_name','redcap_event','race','ethnic_group','site']]
rosetta=pd.read_csv(extrainfo)[['subject','pseudo_guid','nda_interview_date','redcap_event','nda_age','M/F']]

rosetta.nda_interview_date=pd.to_datetime(rosetta.nda_interview_date).dt.strftime('%m/%d/%Y')


In [ ]:
#ksadsslim.columns
#rosetta.columns
hcdtogether2=pd.merge(rosetta,ksadsslim,how='left',on=['subject','redcap_event'])
hcdtogether2.columns[0:25]

hcdtogether2=hcdtogether2.loc[hcdtogether2.redcap_event.isin(eventlist)]
#hcdtogether2.columns[0:25]
#
hcdtogether2=hcdtogether2.rename(columns={"subject":"src_subject_id","M/F":"sex",
                            "nda_age":"interview_age",
                           "nda_interview_date":"interview_date",
                           "pseudo_guid":"subjectkey"}).copy()
#hcdtogether2.columns[0:25]

hcdtogether2=hcdtogether2.drop(columns=['redcap_event','id','patientid','additionalinfo','sitename','k_unusable','k_unusable_specify','common_complete'])
hcdtogether2.columns



In [ ]:
#test=hcdtogether2.copy()
#test=hcdtogether2[['ksadsd1_administered','ksadsd4_administered','ksadsd11_administered']].head(5)
#test.ksadsd1_administered=test.ksadsd1_administered.fillna(-9999).astype(str).str.replace('-9999','')
#test
hcdtogether2.shape

In [ ]:
#remove empties
print("NumRows Before: "+str(hcdtogether2.shape[0]))
subfields=hcdtogether2.columns.to_list()
subfields.remove('subjectkey')
subfields.remove('src_subject_id')
subfields.remove('interview_date')
subfields.remove('interview_age')
subfields.remove('sex')
hcdtogether2=hcdtogether2.dropna(how='all',subset=subfields)
print("NumRows After: "+str(hcdtogether2.shape[0]))


In [ ]:
#set ints
l1=hcdtogether2.columns
l2=['src_subject_id', 'subjectkey', 'interview_date', 'interview_age','sex', 'patienttype']
l3 = [x for x in l1 if x not in l2]

for i in l3:#[:10]:
    print(i)
    try:
        hcdtogether2[i]=hcdtogether2[i].fillna(-9999).astype(str).str.replace('-9999','')
    except:
        print(i,'could not be turned to int')



In [ ]:
#interview_age to int
hcdtogether2.interview_age=hcdtogether2.interview_age.astype(int)
hcdtogether2.to_csv('test.csv',index=False)

In [ ]:

#Write out structure for validation
filePath=pathout+'/hcpksads01.csv'
if os.path.exists(filePath):
    os.remove(filePath)
else:
    print("Can not delete the file as it doesn't exists")

with open(filePath,'a') as f:
    f.write("hcpksads,1\n")
    hcdtogether2.to_csv(f,index=False)
